In [78]:
# DEV SCRIPT - DO NOT USE
import gen_funcs as gf
import ranked_funcs as rf
import clump_funcs as cf
import os
import numpy as np
import re
from scipy.linalg import norm

In [3]:
# Select desired figures
print("Running setup operations", flush=True)
selected_figs = ["old"]
# selected_figs = ["Fig2_Fig4", "Fig6", "Fig7", "Fig8"]

Running setup operations


In [4]:
# Load paths
path_paths = "paths.json"
dict_paths = gf.load_paths_all(path_paths)

(gen_funcs.load_paths_all)
	Reading paths


In [5]:
# Load data
path_data = dict_paths["path_data"]
paths_psliceout = gf.load_paths_psliceout(path_data, selected_figs)
data_all = gf.iteratively_load_data(paths_psliceout)

(gen_funcs.iteratively_load_data)
	Loading data
		Loading file ./data/old/psliceout1.dat


In [6]:
# Rank particles
path_ranked = dict_paths["path_ranked"]
n_neighbors = 100
fout_ranked = gf.iteratively_rank_neighbors(data_all, paths_psliceout, path_ranked, n_neighbors)

(gen_funcs.iteratively_rank_neighbors)
	Ranking particles
		Ranking file ./data/old/psliceout1.dat
(ranked_funcs.rank_neighbors)
	Checking if ranked file exists
		Ranked file ./output/ranked/ranked_100neigh_psliceout1.dat already exists. Skipping.


In [85]:
# Use ranked data to get COM coordinates
print("Using ranked data to get COM coordinates", flush=True)

# Calculate Roche density as threshold
G = 1
Omega_K = 1
H_g = 5.5
threshold_rho = cf.calculate_density_Roche(G, Omega_K, H_g)

# Get COM coordinates for all ranked files
# Setting up shell parameters
dr = 3e-4
n_shells = 5100
threshold_radius = 5.

# Do for all ranked files
data_ranked = {}
coords_com_ranked = {}
for folder in fout_ranked:
    for fin_ranked in fout_ranked[folder]:
        try:
            data_ranked[fin_ranked]
        except KeyError:
            # Consolidate raw data with ranked data
            data_key = re.search("ranked_[0-9]+neigh_(.+).dat", fin_ranked).group(1)
            raw_coords, raw_idx = data_all[data_key]['raw_coords'], data_all[data_key]['raw_idx']

            # Get ranked data
            _data_ranked = rf.read_ranked(fin_ranked, threshold_rho)

            # Get COM coordinates
            _coords_com_ranked = rf.get_com_coords(_data_ranked, dr, n_shells, raw_coords, raw_idx, threshold_radius, var_shell=False)
            data_ranked[fin_ranked] = _data_ranked
            coords_com_ranked[fin_ranked] = _coords_com_ranked

Using ranked data to get COM coordinates
(ranked_funcs.read_ranked)
	Reading in and reducing ranked file ./output/ranked/ranked_100neigh_psliceout1.dat
(ranked_funcs.get_com_coords)
	Finding centres of mass of disconnected clumps
		Iterating through clump 0
		Iterating through clump 1
		Iterating through clump 2
		Iterating through clump 3
		Iterating through clump 4
		Iterating through clump 5
		Iterating through clump 6
		Iterating through clump 7
		Iterating through clump 8
		Iterating through clump 9
		Iterating through clump 10
		Iterating through clump 11
		Iterating through clump 12
		Iterating through clump 13
		Iterating through clump 14
		Iterating through clump 15
		Iterating through clump 16
		Iterating through clump 17
		Iterating through clump 18
		Iterating through clump 19


In [17]:
# distances_from_densest = coords_com_ranked[1:,:3] - coords_com_ranked[0,:3]
distances_from_densest = norm(coords_com_ranked['./output/ranked/ranked_100neigh_psliceout1.dat'][1:,:3] - coords_com_ranked['./output/ranked/ranked_100neigh_psliceout1.dat'][0,:3], axis=1)
threshold_r = (n_neighbors / threshold_rho * 3/4 * 1/np.pi)**1/3
mask = distances_from_densest > threshold_r
# coms = []
# while True:
#     coms.append(coords_com_ranked['./output/ranked/ranked_100neigh_psliceout1.dat'][0,:3])
#     distances_from_densest = distances_from_densest[mask]